# Notebook 2: Process Courses Dataset 
Coursera.csv  
https://www.kaggle.com/datasets/khusheekapoor/coursera-courses-dataset-2021

#### This notebook produces the following data into the _output_datasets_ folder:
```
(COURSE) NODE						course__node.csv
course_id:ID
course_name
course_difficulty_level
course_url
:LABEL = "COURSE”
```
#### Also, it produces intermediate datasets, used for further Skill Matching steps into the _temp_datasets_ folder:
```
(COURSE_SKILL) NODE					courses_skills_TEMP.csv
course_skill_id
course_skill_name

(COURSE_SKILL) RELATION					courses_skills_relationship_TEMP.csv
course_id
course_skill_id
```



## Imports

In [ ]:
%pip install sqlalchemy
%pip install psycopg2-binary
%pip install stanza
%pip install spacy
%pip install nltk
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 4.9 MB/s 
     |████████████████████████████████| 240 kB 30.7 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=3ef88b824bd2bac08eed47804258b7a0d1d88601c58605080005241888f92f8a
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-12-06 21:28:52.546487: E tensorflow/stream_exec

In [ ]:
import pandas as pd
import numpy as np
import stanza
import spacy
import re
stanza.download('en') 
nlp_spacy = spacy.load("en_core_web_sm")
nlp_stanza = stanza.Pipeline('en', processors='tokenize, ner', use_gpu=False, pos_batch_size=3000, download_method=None)

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
# this cell is to support running the notebook in Google Colab

mydrive = ""  # this is when we run locally

# Google Colab:
from google.colab import drive
drive.mount('/content/drive')
mydrive = "/content/drive/MyDrive/DSE 203 — etl/DSE203_Project/"  # this is when we run on COLAB Leslie
mydrive = "/content/drive/MyDrive/DSE203_Project/"  # this is when we run on COLAB Sergey

input_dir = mydrive+"input_datasets/"
output_dir = mydrive+"output_datasets/"
temp_dir = mydrive+"temp_datasets/"

Mounted at /content/drive


## Read Data

In [ ]:
# courses_df = pd.read_csv(input_dir+'coursera_small.csv')
courses_df = pd.read_csv(input_dir+'Coursera.csv')
courses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [ ]:
courses_df.rename(columns = {"Course Name":"course_name",
                     "Course Rating": "course_rating",
                     "Difficulty Level": "course_difficulty_level",
                     "Course URL": "course_url",
                     "Course Description": "course_description",
                     "Skills": "course_skills"
                    }, inplace=True)

In [ ]:
columns_to_leave = ["course_name", 
                    "course_description",
                    "course_difficulty_level",
                    "course_url",
                    "course_skills"]
courses_df = courses_df[columns_to_leave]
courses_df.columns

Index(['course_name', 'course_description', 'course_difficulty_level',
       'course_url', 'course_skills'],
      dtype='object')

## Text Cleaning

In [ ]:
def clean_text(string):
    '''
    remove everything but the alphabetic letters
    '''
    string = re.sub('[^a-zA-Z,.?! ]+', '', string)
    return string


columns_to_clean = ['course_name', 'course_description', 'course_skills']

for column in columns_to_clean:
    courses_df[column] = courses_df[column].apply(clean_text)

### Convert Skills string to list

In [ ]:
courses_df.course_skills = courses_df.course_skills.str.split('  ')
courses_df.head(3)

,course_name,course_description,course_difficulty_level,course_url,course_skills
0,Write A Feature Length Screenplay For Film Or ...,Write a Full Length Feature Film Script In th...,Beginner,https://www.coursera.org/learn/write-a-feature...,"[Drama, Comedy, peering, screenwriting, film, ..."
1,Business Strategy Business Model Canvas Analys...,"By the end of this guided project, you will be...",Beginner,https://www.coursera.org/learn/canvas-analysis...,"[Finance, business plan, persona user experien..."
2,Silicon Thin Film Solar Cells,This course consists of a general presentation...,Advanced,https://www.coursera.org/learn/silicon-thin-fi...,"[chemistry, physics, Solar Energy, film, lambd..."


## Apply NER to extract skills

In [ ]:
def extract_entities_stanza(series):
    '''
    This function will get a dataframe column (series) and will extract skills from the text
    using Stanza library.
    '''
    
    doc = nlp_stanza(series)
    entities_skills = doc.entities
    
    result = list({x.text for x in entities_skills if (x.type == 'ORG') or (x.type == 'PRODUCT')})
    
    return result

In [ ]:
def extract_entities_spacy(series):
    '''
    This function will get a dataframe column (series) and will extract skills from the text
    using Spacy library.
    '''

    doc = nlp_spacy(series, disable=["tok2vec", "parser"])
    entities_skills = doc.ents
    
    result = list({x.text for x in entities_skills if (x.label_ == 'ORG') or (x.label_ == 'PRODUCT')})
    
    return result

In [ ]:
def extend_lists(df):
    '''
    This function will get a Courses dataframe and will combine lists of skills 
    from different columns, will remove duplicates and then will produce a final skills list.
    '''
    # for simplicity
    one  = df.course_name_stanza
    two = df.course_name_spacy
    three = df.description_stanza
    four = df.description_spacy
    five = df.course_skills
    
    # combine into one and eliminate dups
    result = one + two + three + four + five
    
    # lowercase all skills
    result = [x.lower() for x in result]
    
    result = list(set(result))
    
    return result

In [ ]:
%%time

# extract with Stanza
courses_df['course_name_stanza'] = courses_df['course_name'].apply(extract_entities_stanza)
courses_df['description_stanza'] = courses_df['course_description'].apply(extract_entities_stanza)

# extract with Space
courses_df['course_name_spacy'] = courses_df['course_name'].apply(extract_entities_spacy)
courses_df['description_spacy'] = courses_df['course_description'].apply(extract_entities_spacy)

# Combine everything together and remove duplicate skills
courses_df['all_course_skills'] = courses_df.apply(extend_lists,axis=1)

courses_df.head(3)

CPU times: user 1h 55min 57s, sys: 7min 19s, total: 2h 3min 17s
Wall time: 2h 3min 5s


,course_name,course_description,course_difficulty_level,course_url,course_skills,course_name_stanza,description_stanza,course_name_spacy,description_spacy,all_course_skills
0,Write A Feature Length Screenplay For Film Or ...,Write a Full Length Feature Film Script In th...,Beginner,https://www.coursera.org/learn/write-a-feature...,"[Drama, Comedy, peering, screenwriting, film, ...",[],"[Trelby, Active Learning, Celtx, Experiential ...",[],"[IP, Learner Review, Experiential Learning Act...","[dialogue, celtx, creative writing, peering, f..."
1,Business Strategy Business Model Canvas Analys...,"By the end of this guided project, you will be...",Beginner,https://www.coursera.org/learn/canvas-analysis...,"[Finance, business plan, persona user experien...",[],[Business Model Canvas],[Miro],[Business Model Canvas],"[product development, finance, business, perso..."
2,Silicon Thin Film Solar Cells,This course consists of a general presentation...,Advanced,https://www.coursera.org/learn/silicon-thin-fi...,"[chemistry, physics, Solar Energy, film, lambd...",[],"[Photovoltaic Solar Energy, Coursera, Physics ...",[],"[Photovoltaic Solar Energy, Coursera, Physics ...","[chemistry, ecole, electronics, film, thinning..."


## Create relational tables

### Explode table by all skills

In [ ]:
courses_exploded_df = courses_df[['course_name', 'all_course_skills']].explode('all_course_skills')

In [ ]:
courses_exploded_df.columns = ['course_name', 'course_skill']
courses_exploded_df = courses_exploded_df.drop_duplicates(subset=['course_name','course_skill'])
courses_exploded_df.reset_index(inplace=True)
courses_exploded_df.columns = ['course_id', 'course_name', 'course_skill']
courses_exploded_df[courses_exploded_df.course_skill.str.contains('ython')]

,course_id,course_name,course_skill
231,16,Python Programming Essentials,python programming
234,16,Python Programming Essentials,python
240,16,Python Programming Essentials,python syntax and semantics
545,40,Realtime OCR and Text Detection with Tensorflo...,python programming
769,57,Prediction and Control with Function Approxima...,python
...,...,...,...
48180,3512,Mining Data to Extract and Visualize Insights ...,python programming
48184,3512,Mining Data to Extract and Visualize Insights ...,python
48263,3517,"Capstone Retrieving, Processing, and Visualizi...",python programming
48266,3517,"Capstone Retrieving, Processing, and Visualizi...",python


## Create table of course skills (without duplicates)

In [ ]:
courses_skills_df = courses_exploded_df[['course_skill']].copy()
courses_skills_df = courses_skills_df.drop_duplicates() \
               .reset_index(drop=True) \
               .reset_index() \
               .rename(columns={'course_skill':'course_skill_name', 'index':'course_skill_id'})
courses_skills_df.course_skill_id = courses_skills_df.course_skill_id.astype('int')

courses_skills_df[courses_skills_df.course_skill_name.str.contains('ython')]

,course_skill_id,course_skill_name
204,204,python programming
207,207,python
213,213,python syntax and semantics
1716,1716,python programming informationtechnology suppo...
1874,1874,pythonflask
...,...,...
13149,13149,python imputations
13249,13249,"python, gradient descent"
13376,13376,python for statistical data visualization and ...
13447,13447,ipython computerscience softwaredevelopment


In [ ]:
courses_skills_df.to_csv(temp_dir+"courses_skills_TEMP.csv", index=False)

## Map course skills back to the courses to for creating the intermedite (temp) relationships table

In [ ]:
courses_exploded_df = courses_exploded_df.merge(courses_skills_df, how='outer', left_on='course_skill', right_on='course_skill_name')
courses_exploded_df

,course_id,course_name,course_skill,course_skill_id,course_skill_name
0,0,Write A Feature Length Screenplay For Film Or ...,dialogue,0,dialogue
1,330,"Sren Kierkegaard Subjectivity, Irony and the ...",dialogue,0,dialogue
2,1906,Becoming Part of the Globalised Workplace,dialogue,0,dialogue
3,2424,Writing Stories About Ourselves,dialogue,0,dialogue
4,2445,ART of the MOOC Activism and Social Movements,dialogue,0,dialogue
...,...,...,...,...,...
48331,3518,Patrick Henry Forgotten Founder,washingtons,16756,washingtons
48332,3519,Business intelligence and data analytics Gener...,generate,16757,generate
48333,3520,Rigid Body Dynamics,rigid body dynamics,16758,rigid body dynamics
48334,3520,Rigid Body Dynamics,d rig,16759,d rig


In [ ]:
courses_exploded_df[["course_id", "course_skill_id"]].to_csv(temp_dir+"courses_skills_relationship_TEMP.csv",index=False)

### (node) COURSE

In [ ]:
columns_to_leave = ["course_name", 
                    "course_difficulty_level",
                    "course_url"]
courses_df = courses_df[columns_to_leave]
courses_df[":LABEL"] = "COURSE"
courses_df

<ipython-input-19-9a38d311a344>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  courses_df[":LABEL"] = "COURSE"


,course_name,course_difficulty_level,course_url,:LABEL
0,Write A Feature Length Screenplay For Film Or ...,Beginner,https://www.coursera.org/learn/write-a-feature...,COURSE
1,Business Strategy Business Model Canvas Analys...,Beginner,https://www.coursera.org/learn/canvas-analysis...,COURSE
2,Silicon Thin Film Solar Cells,Advanced,https://www.coursera.org/learn/silicon-thin-fi...,COURSE
3,Finance for Managers,Intermediate,https://www.coursera.org/learn/operational-fin...,COURSE
4,Retrieve Data using SingleTable SQL Queries,Beginner,https://www.coursera.org/learn/single-table-sq...,COURSE
...,...,...,...,...
3517,"Capstone Retrieving, Processing, and Visualizi...",Beginner,https://www.coursera.org/learn/python-data-vis...,COURSE
3518,Patrick Henry Forgotten Founder,Intermediate,https://www.coursera.org/learn/henry,COURSE
3519,Business intelligence and data analytics Gener...,Advanced,https://www.coursera.org/learn/business-intell...,COURSE
3520,Rigid Body Dynamics,Beginner,https://www.coursera.org/learn/rigid-body-dyna...,COURSE


In [ ]:
courses_df.to_csv(output_dir+"course__node.csv", index_label="course_id:ID")